In [1]:
from numpy  import *
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import random as random
random.seed(102)

In [193]:
def WhichGrid(x,y,XRange,YRange,XNum,YNum):
    G=floor(x*XNum/XRange)*YNum + floor(y*YNum/YRange)+1
    return G
#Calculates which grid a particular location is in


In [192]:
def Kernel(dist_squared):
    P=[-9.2123e-5,9.5628e-4,3.3966e-3,-3.3687e-2,-1.30519e-1, -0.609262, -3.231772]
    K=np.exp(np.polyval(P,dist_squared))
    if (dist_squared<0.0138):
        K=0.3093
    if (dist_squared>60*60):
        K=0
    return K

In [304]:
#Iteration Space
def Iterate(Status, x, y, Suscept, Transmiss, RingCull, grid, first_in_grid, last_in_grid, Num, MaxRate):

    Event=0*Status;

    [INF]=np.where((Status>5) & (Status<12));  NI=len(INF); #Note reported farms still infectious
    IGrids=grid[INF];

    for i in range(NI):
        INFi=INF[i];
        MaxProb=1-np.exp(-Transmiss[INFi]*Num*MaxRate[int(IGrids[i]),:]);
        [m]=np.where(MaxProb-np.random.rand(int(max(grid)))>0);  #these are grids that need further consideration

        for n in range(len(m)):
            s=1;
            M=m[n];
            PAB=1-np.exp(-Transmiss[INFi]*MaxRate[int(IGrids[i]),M]);
            if (PAB==1):
                ind=np.arange(first_in_grid[M],last_in_grid[M])
                Q=1-np.exp(-Transmiss[INFi]*Suscept[ind]*Kernel((x[INFi]-x[ind])**2+(y[INFi]-y[ind])**2));
                Event[ind[(np.random.rand(np.size(Q))-Q<0) & (Status[ind]==0)]]=1;
            else:
                for j in range(int(Num[M])):
                    ind=first_in_grid[M]+j-1;
                    P=1-s*(1-PAB)**(Num[M]+1-j);
                    R=np.random.rand(1);
                    if (R<PAB/P):
                        s=0;
                        Q=1-np.exp(-Transmiss[INFi]*Suscept[ind]*Kernel((x[INFi]-x[ind])**2+(y[INFi]-y[ind])**2));
                        if (R<Q/P and Status[ind]==0):
                            Event[ind]=1;

    [m]=np.where(Status>0);
    Status[m]=Status[m]+1;
    Status=Status+Event;

    [m]=np.where(Status==13); #Initiate Ring Culling Around Reported Farm
    for i in range(len(m)):
        Status[m[i]]=-1;
        D=(x[m[i]]-x[:])**2+(y[m[i]]-y[:])**2;
        [n]=np.where(D<RingCull**2);
        Status[n]=-1;
        
    return Status


In [220]:
N=4000;
Size=20;
Y0=1;
RingCull=0;
MaxTime=1000;
#may need to check these are valid parameters

In [287]:
x=Size*np.random.rand(N)
y=Size*np.random.rand(N)
Status=np.zeros(N)
#Status[0:Y0-1]=1
Status[0]=1
R=np.random.rand(N)
Cows=np.zeros(N)
Sheep=np.zeros(N)
[n]=np.where(R<0.73)
[m]=np.where((R<0.46) | (R>0.73))
while (len(m)+len(n)>0):
    Sheep[m]=350*np.exp(np.random.rand(len(m)));
    Cows[n]=70*np.exp(np.random.rand(len(n)));
    m=m[(Sheep[m]<1) | (Sheep[m]>30000)];
    n=n[(Cows[n]<1) | (Cows[n]>5000)];

Suscept=Sheep+10.5*Cows; Transmiss=5.1e-7*Sheep + 7.7e-7*Cows

In [288]:
grid=WhichGrid(x,y,Size,Size,10,10)
tmp=np.sort(grid)
i=np.argsort(grid)
x=x[i]
y=y[i]
Status=Status[i]
grid=grid[i]
Xgrid=np.zeros(int(max(grid)));Ygrid=np.zeros(int(max(grid)))
Num=np.zeros(int(max(grid)));first_in_grid=np.zeros(int(max(grid)))
last_in_grid=np.zeros(int(max(grid)));Max_Sus_grid=np.zeros(int(max(grid)))

for i in range(int(max(grid))):
    Xgrid[i]=floor((i-1)/10)
    Ygrid[i]=mod((i-1),10)
    [m]=np.where(grid==i)
    Num[i]=len(m)
    if (Num[i]>0):
        first_in_grid[i]=min(m)
        last_in_grid[i]=max(m)
        Max_Sus_grid[i]=max(Suscept[m])
    else:
        first_in_grid[i]=0
        last_in_grid[i]=-1
        Max_Sus_grid[i]=0


In [289]:
#Work out grid to maximum grid transmission probabilities
max_grid=int(max(grid))
MaxRate=np.zeros((max_grid,max_grid))
Distsq=[]
for i in range(max_grid):
    for j in range(max_grid):
        if ((i==j) | (Num[i]==0) | (Num[j]==0)):
            MaxRate[i,j]=inf;
        else:
            
            #Dist2=(Size*(abs(Xgrid[i]-Xgrid[j])/10))**2 + (Size*(abs(Ygrid[i]-Ygrid[j])/10))**2
            
            Dist2=(Size*max([0,(abs(Xgrid[i]-Xgrid[j])-1)])/10)**2 + (Size*max([0,(abs(Ygrid[i]-Ygrid[j])-1)])/10)**2
            Distsq.append(Dist2)
            MaxRate[i,j]=np.multiply(Max_Sus_grid[j],Kernel(Dist2))

In [305]:
i=0;t = np.zeros(MaxTime+100);t[i]=0;
S = np.zeros(MaxTime+100)
E = np.zeros(MaxTime+100)
I = np.zeros(MaxTime+100)
R = np.zeros(MaxTime+100)
R2 = np.zeros(MaxTime+100)
CullSheep = np.zeros(MaxTime+100)
CullCattle = np.zeros(MaxTime+100)


S[i]=len(Status[Status==0]);time=0
E[i]=len(Status[(Status>0)& (Status<=5)])
I[i]=len(Status[(Status>5)& (Status<=9)])
R[i]=len(Status[Status==10])
R2[i]=len(Status[Status>9])
CullSheep[i]=0
CullCattle[i]=0
i=i+1
IterateFlag=1
while ((time<MaxTime) and (IterateFlag==1)):
    Status=Iterate(Status, x, y, Suscept, Transmiss, RingCull, grid, first_in_grid, last_in_grid, Num, MaxRate)
    [Sus]=np.where(Status==0)
    [Exp]=np.where((Status>0) & (Status<=5))
    [Inf]=np.where((Status>5) & (Status<=9))
    [Rep]=np.where(Status>9)
    [Culled]=np.where(Status<0)
    S[i]=len(Sus); E[i]=len(Exp); I[i]=len(Inf);
    R[i]=len(Status[Status==10])
    R2[i]=len(Rep)
    CullSheep[i]=sum(Sheep[Culled]);CullCattle[i]=sum(Cows[Culled])
    t[i]=t[i-1]+1;i=i+1
    time+=1
    if (t[-1]>5):
        if (E[-4]+I[-4]+R2[-4]==0):
            IterateFlag=0
            
    xculledplot = [x[i] for i in Culled]
    xsusplot = [x[i] for i in Sus]
    xexpplot = [x[i] for i in Exp]
    xinfplot = [x[i] for i in Inf]
    xrepplot = [x[i] for i in Rep]
    
    yculledplot = [y[i] for i in Culled]
    ysusplot = [y[i] for i in Sus]
    yexpplot = [y[i] for i in Exp]
    yinfplot = [y[i] for i in Inf]
    yrepplot = [y[i] for i in Rep]
    
    plt.scatter(xculledplot, yculledplot)
    plt.scatter(xsusplot, ysusplot)
    plt.scatter(xexpplot, yexpplot)
    plt.scatter(xinfplot, yinfplot)
    plt.scatter(xrepplot, yrepplot)
    plt.pause(0.05)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in multiply
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in greater
  if sys.path[0] == '':


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

NameError: name 'ind' is not defined

In [61]:
import pandas as pd

In [233]:
np.where(Status==1)

(array([3726]),)

In [218]:
#pd.read_csv("movements_unique")

In [68]:
data=pd.read_csv("Farms_2010_RSG",header=None, delim_whitespace=True)

In [71]:
data.values[:,0]

array([  1.,   1.,   1., ...,  98.,  98.,  98.])

In [72]:
movement_data=pd.read_csv("movements_unique",header=None, delim_whitespace=True)

In [74]:
farmnum_from=movement_data.values[:,0]
farmnum_to=movement_data.values[:,1]
weight_link=movement_data.values[:,2]

In [77]:
G=nx.read_weighted_edgelist("movements_unique")

In [76]:
import networkx as nx

In [121]:
a=np.random.randint(0,10,8)